# InThon Baseline Code

## 1. 라이브러리 설치
필요한 라이브러리를 설치합니다.

먼저, 이 코드는 Hugging Face의 `peft`(Parameter-Efficient Fine-Tuning) 라이브러리를 활용하여 모델 파인튜닝을 합니다. 이를 통해 큰 모델을 메모리 효율적으로 파인튜닝할 수 있습니다. `bitsandbytes`는 모델을 저비트 수 형식(예: 4비트)으로 변환하여 메모리 사용량을 줄입니다.

In [ ]:
!pip install -q git+https://github.com/huggingface/peft.git transformers bitsandbytes datasets
!pip install git+https://github.com/salaniz/pycocoevalcap

import torch
from PIL import Image
import requests
from transformers import Blip2Processor, Blip2ForConditionalGeneration, InstructBlipProcessor, InstructBlipForConditionalGeneration, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.utils.data import DataLoader

## 2. 환경 및 디바이스 설정
**PyTorch**와 **CUDA**를 사용하여 훈련에 사용할 디바이스를 확인합니다.

여기서 `torch.device`를 통해 모델을 `cuda`(GPU)에서 실행할지 `cpu`에서 실행할지를 결정합니다. GPU는 병렬 연산을 빠르게 처리할 수 있기 때문에, 가능하면 GPU를 사용하는 것이 학습 속도에 유리합니다.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"사용 중인 디바이스: {device}")

## 3. 모델 및 프로세서 설정
`BitsAndBytesConfig`를 사용하여 4비트 양자화를 설정하고, `Blip2Processor`와 `Blip2ForConditionalGeneration`을 설정합니다.

일반적으로 모델 파라미터는 32비트로 저장되지만, 이를 4비트로 줄이면 메모리 사용량이 크게 감소합니다. `Blip2Processor`는 이미지와 텍스트를 같이 처리하는 프로세서로, 입력 데이터를 모델이 이해할 수 있는 형태로 변환해 줍니다.

### 양자화 (Quantization)
**4비트, 8비트 양자화**는 모델의 각 숫자(파라미터)를 4비트로 저장하여 메모리를 절약하는 방법입니다. 모델의 성능에 약간의 손실이 있을 수 있지만, 많은 경우 큰 차이가 없습니다.

#### 양자화 원리
양자화는 부동소수점 수(floating point)로 표현된 파라미터를 더 적은 비트의 정수(integer)로 근사하여 표현하는 방식입니다. 이때, 수의 범위가 줄어들기 때문에 정밀도가 다소 감소할 수 있지만, 모델의 효율성은 크게 향상됩니다.

1. **연속 값에서 이산 값으로 변환**:
   주어진 범위 내에서 연속적인 값들을 정해진 개수의 **이산 값(discrete values)**으로 변환합니다. 예를 들어, 4비트 양자화에서는 $2^4 = 16$개의 이산 값으로 모든 파라미터 값을 근사하게 됩니다.

2. **스케일링 (Scaling)**:
   원래의 실수 값 $x$를 정수 값 $q$로 변환하기 위해, 스케일링 인자 $s$와 오프셋 $z$를 사용하여 양자화를 수행합니다. 변환 수식은 다음과 같습니다.

   $$
   q = \text{round}\left(\frac{x}{s} + z\right)
   $$

   여기서:
   - $s$는 스케일링 인자(양자화된 값의 크기를 조절).
   - $z$는 오프셋 값으로, 일반적으로 0이나 양자화 범위의 중앙값으로 설정됩니다.
   - $\text{round}$는 반올림 함수로, 실수 값을 가장 가까운 정수 값으로 변환합니다.

3. **복원 (Dequantization)**:
   양자화된 값을 다시 원래의 값에 가깝게 복원하려면 다음과 같은 복원식을 사용합니다.

   $$
   x \approx s \cdot (q - z)
   $$

   이를 통해 양자화된 정수 값을 부동소수점 값으로 다시 근사할 수 있으며, 복원된 값은 원래 값과 약간의 차이가 있을 수 있지만, 모델 성능에 큰 영향을 주지 않도록 설계합니다.

4. **정밀도와 메모리 효율의 균형**:
   양자화는 모델 크기와 메모리 사용을 줄이는 데 효과적이며, 특히 저비트 양자화(예: 4비트)는 모델을 더 작고 빠르게 만듭니다. 4비트 양자화로 인해 수치의 정밀도가 약간 감소하지만, 학습 중 적절한 조정이 이루어지면 성능 저하를 최소화할 수 있습니다.

이와 같은 4비트 양자화 기법을 통해 모델의 메모리 효율성을 극대화하면서도, 예측 성능을 유지할 수 있습니다.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    # load_in_4bit=True,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16
)

processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-flan-t5-xl")
model = InstructBlipForConditionalGeneration.from_pretrained(
    "Salesforce/instructblip-flan-t5-xl",
    quantization_config=bnb_config,
    # device_map="auto"
)

## 4. 모델 준비 및 LoRA 설정
`prepare_model_for_kbit_training`로 모델을 준비하고, **LoRA** 파인튜닝을 위한 설정을 합니다.

### LoRA (Low-Rank Adaptation)

**LoRA**는 큰 모델을 파인튜닝할 때 모든 파라미터를 조정하면 메모리가 많이 필요하므로, 모델의 일부 중요한 파라미터만 선택적으로 학습하여 메모리 효율성을 높이는 기법입니다.

#### 이론 및 수식
LoRA는 모델의 가중치 행렬 $W \in \mathbb{R}^{d \times k}$을 두 개의 저랭크 행렬 $A \in \mathbb{R}^{d \times r}$과 $B \in \mathbb{R}^{r \times k}$로 근사합니다. 여기서 $r$은 저랭크 차원(rank)을 의미하며, $r$이 작을수록 메모리 절약 효과가 큽니다.

1. **가중치 행렬 분해**:
   $W$를 LoRA로 근사하기 위해 다음과 같이 표현합니다.
   
   $$
   W \approx W_0 + \Delta W = W_0 + A B
   $$
   
   여기서:
   - $W_0$는 기존의 고정된 원본 가중치입니다.
   - $\Delta W = A B$는 학습 가능한 저랭크 행렬의 곱으로, 파인튜닝 시 학습됩니다.

2. **저랭크 근사**:
   저랭크 근사에서 $A$와 $B$의 차원을 $d \times r$ 및 $r \times k$로 설정하면, 전체 파라미터 수는 $d \times k$에서 $d \times r + r \times k$로 줄어듭니다. 즉, 전체 파라미터 수는 다음과 같습니다.
   
   $$
   \text{파라미터 수} = d \times r + r \times k
   $$

   이는 원래 파라미터 수 $d \times k$에 비해 훨씬 적은 파라미터를 사용하여 근사할 수 있음을 의미합니다.

3. **파라미터 효율성**:
   LoRA의 저랭크 차원 $r$이 작을수록 메모리 효율성이 높아집니다. 예를 들어 $r$을 $d$나 $k$에 비해 매우 작은 값으로 설정하면, 파라미터 수가 크게 줄어듭니다.

이와 같은 LoRA 기법을 통해, 모델의 일부 주요 파라미터를 저랭크 행렬로 근사하여 메모리 사용량을 줄이면서도 파인튜닝 성능을 유지할 수 있습니다.

LoRA: https://arxiv.org/abs/2106.09685


In [ ]:
for name, module in model.named_modules():
    print(name)

In [ ]:
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q", "k", "v", "o"],
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_2_SEQ_LM",
    # task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
print("LoRA 파인튜닝 설정 완료.")

## 5. 학습 가능한 파라미터 확인
전체 파라미터 중 학습 가능한 파라미터와 비율을 확인합니다.

> 이 부분에 대한 코드는 수정해서는 안 됩니다! 수정을 진행하였을 시 평가에 영향이 있을 수 있습니다. 채점 기준 중 하나인 **학습 가능한 파라미터의 수**와 직결되는 부분입니다. 이와 관련된 부분을 허위로 기재한 것이 적발될 시 평가에 불이익이 있을 수 있습니다.



In [ ]:
def print_trainable_params(model):
    trainable_params = 0
    all_params = 0
    for _, param in model.named_parameters():
        all_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(f"전체 파라미터 수: {all_params / 1e6:.2f}M")
    print(f"학습 가능한 파라미터 수: {trainable_params}")
    print(f"파라미터 비율: {100 * trainable_params / all_params:.2f}%")

print_trainable_params(model)

## 6. 데이터셋 로드 및 분할
데이터셋을 로드하고 학습, 검증, 테스트 세트로 나눕니다.

> 이 부분에 대한 코드는 수정해서는 안 됩니다! 수정을 진행하였을 시 평가에 영향이 있을 수 있습니다. 채점 기준 중 하나인 **사용된 데이터셋의 크기**와 직결되는 부분입니다. 이와 관련된 부분을 허위로 기재한 것이 적발될 시 평가에 불이익이 있을 수 있습니다.

In [ ]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader, random_split
import random

dataset_path = os.path.join('/content', 'dataset.csv')
data_df = pd.read_csv(dataset_path)

train_df = data_df[data_df['train'] == True]
val_df = data_df[data_df['val'] == True]
test_df = data_df[data_df['test'] == True]
print(f"Training set size: {len(train_df)}, Validation set size: {len(val_df)}, Test set size: {len(test_df)}")

num_epochs = 2
print(f"Total training data point size: {len(train_df) * num_epochs}")

## 7. 커스텀 데이터셋 클래스 및 DataLoader 생성

### 개념 설명
이 단계에서는 `CustomImageCaptionDataset` 클래스를 정의하여 데이터셋을 PyTorch 형식으로 로드합니다. 각 샘플을 이미지와 텍스트 형태로 받아 모델의 입력 형식에 맞게 전처리합니다.

### 이론 및 수식
텍스트는 정수 인코딩을 통해 모델 입력으로 들어가고, 이미지 데이터는 픽셀 값이 `[0, 1]` 범위로 정규화됩니다.

1. **텍스트 인코딩**:
   $
   \text{input_ids} = \text{tokenizer(text)}
   $
   
   `inputs['input_ids']`는 텍스트를 정수 형태로 변환한 결과입니다.

2. **이미지 정규화**:
   $
   \text{pixel_values} = \frac{\text{image} - \text{min(image)}}{\text{max(image)} - \text{min(image)}}
   $

   이 정규화는 모델이 입력 픽셀값(`inputs['pixel_values']`)을 일정한 범위 내에서 처리할 수 있게 하여 학습의 안정성을 높입니다.


In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import requests
from io import BytesIO

class CustomImageCaptionDataset(Dataset):
    def __init__(self, df, processor, test=False):
        self.df = df
        self.processor = processor
        self.test = test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if not self.test:
            image_url = self.df.iloc[idx]["url"]
            image_id = self.df.iloc[idx]["Image_ID"]
            caption = self.df.iloc[idx]["Paragraph"]

            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')

            inputs = self.processor(
                images=image,
                text=caption,
                padding="max_length",
                truncation=True,
                max_length=256,
                return_tensors="pt"
            )
            inputs = {k: v.squeeze(0) for k, v in inputs.items()}
            inputs['labels'] = inputs['input_ids'].clone()

            inputs['image_url'] = image_url
            inputs['Image_ID'] = image_id
            inputs['reference_caption'] = caption

        else:
            image_url = self.df.iloc[idx]["url"]
            image_id = self.df.iloc[idx]["Image_ID"]

            inputs = dict()
            inputs['image_url'] = image_url
            inputs['Image_ID'] = image_id

        return inputs

In [ ]:
train_dataset = CustomImageCaptionDataset(train_df, processor)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)

val_dataset = CustomImageCaptionDataset(val_df, processor)
val_dataloader = DataLoader(val_dataset, shuffle=False, batch_size=8)

test_dataset = CustomImageCaptionDataset(test_df, processor, test=True)
test_dataloader = DataLoader(test_dataset, shuffle=False, batch_size=8)

## 8. 옵티마이저와 스케줄러 설정

### 개념 설명
**옵티마이저**는 모델 파라미터를 조정하는 방법을 정의합니다. **AdamW**는 Adam 옵티마이저의 변형으로, L2 정규화 대신 가중치 감쇠(Weight Decay)를 적용하여 과적합을 방지합니다. **스케줄러**는 학습 중 학습률을 동적으로 조정하여 학습 효율을 높입니다.

### 이론 및 수식
1. **AdamW 옵티마이저**:
   Adam 옵티마이저는 모멘텀을 사용하는 SGD 방식입니다. 각 파라미터에 대해 다음과 같은 방식으로 업데이트가 이루어집니다.

   $
   m_t = \beta_1 \cdot m_{t-1} + (1 - \beta_1) \cdot g_t
   $

   $
   v_t = \beta_2 \cdot v_{t-1} + (1 - \beta_2) \cdot g_t^2
   $

   $
   \hat{m}_t = \frac{m_t}{1 - \beta_1^t}, \quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}
   $

   $
   \theta_{t+1} = \theta_t - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} - \eta \lambda \theta_t
   $

   여기서 $( \theta )$는 모델 파라미터, $( g_t )$는 현재 그라디언트, $( \lambda )$는 가중치 감쇠 계수입니다.

2. **스케줄러**: 학습률 감소를 위한 스케줄러는 StepLR을 사용합니다. 이 스케줄러는 주어진 스텝마다 학습률을 감소시킵니다.

   $
   \eta_{t+1} = \gamma \cdot \eta_t
   $

   여기서 $( \gamma )$는 감소율입니다.

In [ ]:
from transformers import get_linear_schedule_with_warmup

for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True

optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=5e-5)
total_steps = len(train_dataloader) * num_epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=total_steps // 2, gamma=0.1)

## 9. 학습 루프

### 1. Autoregressive 모델
여기 사용된 모델은 **autoregressive** 모델로, 각 타임 스텝에서 이전 타임 스텝의 출력을 바탕으로 다음 토큰을 순차적으로 예측합니다. 이 방식은 자연어 생성에서 주로 쓰이며, 이미지 캡션 생성 작업에서도 각 단어가 이전 단어와의 관계를 통해 예측됩니다.

### 2. 교차 엔트로피 손실 (Cross-Entropy Loss)
손실 함수로는 **교차 엔트로피 손실**(Cross-Entropy Loss)을 사용합니다. 모델이 예측한 확률 분포와 실제 정답 레이블 간의 차이를 측정하여, 모델의 예측이 실제 정답과 얼마나 일치하는지 평가합니다.

#### 수식
주어진 입력 시퀀스 $x$에 대해 모델이 각 시간 단계 $t$에서 다음 토큰 $y_t$를 예측하도록 학습하는 과정에서 교차 엔트로피 손실 $L$은 다음과 같이 정의됩니다.

$L = -\sum_{t=1}^{T} \log p(y_t | x)$

여기서:
- $T$는 시퀀스의 길이입니다.
- $y_t$는 $t$번째 타임 스텝에서의 실제 정답 토큰입니다.
- $p(y_t | x)$는 모델이 $x$를 입력받았을 때 $y_t$ 토큰을 예측할 확률입니다.

이 수식에서 모델은 각 토큰의 예측 확률이 실제 정답 토큰에 가까워질수록 손실이 작아지며, 모델이 더 정확해집니다.

### 3. Softmax 함수와 Temperature 조정
모델은 각 타임 스텝에서 다음 토큰의 확률을 예측할 때 **softmax** 함수를 사용하여 출력 확률 분포를 만듭니다.

#### Softmax 함수
Softmax는 벡터 $z = (z_1, z_2, \dots, z_n)$를 입력받아 각 요소의 확률을 계산합니다.

$p(y_t | x) = \frac{\exp(z_t / T)}{\sum_{i=1}^{n} \exp(z_i / T)}$

여기서:
- $z_i$는 모델의 출력 로짓(logit)입니다.
- $T$는 temperature 값으로, softmax의 **temperature**를 조정해 확률 분포의 집중도를 제어할 수 있습니다.

#### Temperature의 역할
- $T = 1$: 기본적인 softmax로, 확률 분포의 집중도에 변화가 없습니다.
- $T > 1$: 확률 분포를 평탄화시켜 더 다양한 토큰이 선택될 가능성을 높여 **탐색적** 예측을 가능하게 합니다.
- $T < 1$: 확률 분포를 더 예리하게 만들어 가장 높은 확률을 가진 토큰이 더 자주 선택되며 **보수적** 예측을 가능하게 합니다.

이렇게 softmax와 temperature 조정은 모델이 생성하는 문장의 다양성을 조절하는 데 중요한 역할을 합니다.

### 4. 코드 내 손실 누적 및 그라디언트 업데이트
이 코드에서는 `accumulation_steps`를 통해 여러 미니배치의 손실을 누적하여 메모리 사용량을 조절합니다. 손실은 다음과 같이 누적됩니다.

1. **손실 누적**:
   $L_{\text{acc}} = \sum_{i=1}^{S} L_i$
   여기서 $S$는 `accumulation_steps` 값입니다.

2. **손실의 평균 계산**:
   최종적으로 누적 손실을 $S$로 나눈 평균 손실로 그라디언트를 계산하여 역전파합니다.
   $\bar{L} = \frac{L_{\text{acc}}}{S} = \frac{1}{S} \sum_{i=1}^{S} L_i$

3. **역전파 및 파라미터 업데이트**:
   손실 $\bar{L}$에 대해 그라디언트를 계산하고 역전파를 수행하여 파라미터를 업데이트합니다.

이 과정을 통해 모델이 예측 성능을 점차적으로 개선해 나갑니다.

In [ ]:
import torch
from types import MethodType

def new_forward(self, *args, **kwargs):
    if 'inputs_embeds' in kwargs:
        kwargs.pop('inputs_embeds')
    return self.base_model.forward(*args, **kwargs)

model.forward = MethodType(new_forward, model)

def forward_pass(input_ids, pixel_values, qformer_input_ids, qformer_attention_mask, labels):
    outputs = model(
        input_ids=input_ids,
        pixel_values=pixel_values,
        qformer_input_ids=qformer_input_ids,
        qformer_attention_mask=qformer_attention_mask,
        labels=labels
    )
    logits = outputs.logits

    if logits.size(1) > labels.size(1):
        logits = logits[:, :labels.size(1), :]
    elif logits.size(1) < labels.size(1):
        padding = torch.zeros((logits.size(0), labels.size(1) - logits.size(1), logits.size(2)), device=logits.device)
        logits = torch.cat([logits, padding], dim=1)

    loss_fct = torch.nn.CrossEntropyLoss()
    loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
    return loss

In [ ]:
from tqdm import tqdm

accumulation_steps = 4

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    print(f"Epoch {epoch+1} / {num_epochs}")

    with tqdm(total=len(train_dataloader), desc=f"Epoch {epoch+1} (Training)", postfix={'Train Loss': 0.0}) as pbar:
        for idx, batch in enumerate(train_dataloader):
            instruction_text = ["Describe this image in detail."] * batch["pixel_values"].size(0)

            pixel_values = (batch["pixel_values"] - batch["pixel_values"].min()) / (batch["pixel_values"].max() - batch["pixel_values"].min())
            inputs = processor(
                images=pixel_values,
                text=instruction_text,
                return_tensors="pt",
                padding=True
            ).to(device)

            labels = batch["labels"].to(device)

            loss = forward_pass(
                inputs["input_ids"],
                inputs["pixel_values"],
                inputs.get("qformer_input_ids", None),
                inputs.get("qformer_attention_mask", None),
                labels
            ) / accumulation_steps

            loss.backward()
            epoch_loss += loss.item()

            if (idx + 1) % accumulation_steps == 0 or (idx + 1 == len(train_dataloader)):
                optimizer.step()
                optimizer.zero_grad()
                torch.cuda.empty_cache()

            pbar.set_postfix({'Train Loss': epoch_loss / (idx + 1)})
            pbar.update(1)

    scheduler.step()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        with tqdm(total=len(val_dataloader), desc=f"Epoch {epoch+1} (Validation)", postfix={'Val Loss': 0.0}) as pbar:
            for idx, batch in enumerate(val_dataloader):
                instruction_text = ["Describe this image in detail."] * batch["pixel_values"].size(0)

                pixel_values = (batch["pixel_values"] - batch["pixel_values"].min()) / (batch["pixel_values"].max() - batch["pixel_values"].min())
                inputs = processor(
                    images=pixel_values,
                    text=instruction_text,
                    return_tensors="pt",
                    padding=True
                ).to(device)

                labels = batch["labels"].to(device)

                loss = forward_pass(
                    inputs["input_ids"],
                    inputs["pixel_values"],
                    inputs.get("qformer_input_ids", None),
                    inputs.get("qformer_attention_mask", None),
                    labels
                )
                val_loss += loss.item()

                pbar.set_postfix({'Val Loss': val_loss / (idx + 1)})
                pbar.update(1)

    avg_train_loss = epoch_loss / len(train_dataloader)
    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Epoch {epoch+1} completed | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

torch.cuda.empty_cache()

## 10. 평가 및 메트릭 계산
테스트 데이터셋에 대해 **SPICE, CLIPScore, CHAIRf** 메트릭을 계산합니다.

**SPICE**: 텍스트의 질을 평가하는 점수로, 이미지 캡션의 내용과 관련된 객체, 관계, 속성 등을 기반으로 측정합니다.

**CLIPScore**: CLIP 모델을 사용해 이미지와 텍스트의 유사도를 측정하는 점수입니다.

**CHAIR**: 캡션이 이미지와 얼마나 잘 일치하는지 평가하는 지표입니다. 특히, 캡션에 어떤 객체가 포함되었는지를 검토하는 데 초점을 둡니다. Precision의 성격이 강한 CHAIRi, CHAIRs와 달리 CHAIRf는 Recall의 성격도 같이 고려하는 평가 지표입니다.

SPICE: https://arxiv.org/abs/1607.08822

CLIPScore: https://arxiv.org/abs/2104.08718

CHAIR: https://arxiv.org/abs/1809.02156



In [ ]:
from transformers import CLIPProcessor, CLIPModel
import torch
from tqdm import tqdm
import requests
from PIL import Image
from io import BytesIO
from pycocoevalcap.spice.spice import Spice
from collections import Counter
import spacy

evaluation_objects = [
    "man", "woman", "tree", "sky", "building", "window", "shirt", "wall",
    "sign", "grass", "water", "table", "train", "plate", "car", "dog", "cat",
    "giraffe", "light", "pole", "plane", "boy", "zebra", "bus", "elephant",
    "ground", "hair", "girl", "horse", "cloud", "hand", "clock", "people",
    "snow", "bird", "chair", "fence", "glass", "floor", "bear", "boat",
    "street", "head", "door", "road", "shoe", "leg", "eye", "hat"
]

nlp = spacy.load("en_core_web_sm")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
spice_scorer = Spice()

def get_singular_form(word):
    return word.lemma_

def extract_objects_from_caption(caption, object_list):
    doc = nlp(caption)
    objects_in_caption = set()

    for token in doc:
        word = token.lemma_.lower()
        if word in object_list:
            objects_in_caption.add(word)

    return objects_in_caption

def calculate_chair_metrics(generated_caption, image_objects):
    caption_objects = extract_objects_from_caption(generated_caption, evaluation_objects)

    hallucinated_objects = caption_objects - set(image_objects)
    missing_objects = set(image_objects) - caption_objects
    true_positives = caption_objects & set(image_objects)

    precision = len(true_positives) / (len(true_positives) + len(hallucinated_objects)) if len(caption_objects) > 0 else 0
    recall = len(true_positives) / (len(true_positives) + len(missing_objects)) if len(image_objects) > 0 else 0

    chairf = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    print(f"환각 객체: {hallucinated_objects}, 누락 객체: {missing_objects}, 캡션 객체: {caption_objects}, 이미지 객체: {image_objects}")

    return chairf

def calculate_metrics(image, generated_caption, reference_caption, image_objects):
    spice_score, _ = spice_scorer.compute_score({0: [reference_caption]}, {0: [generated_caption]})

    inputs = clip_processor(text=generated_caption,
                            images=image,
                            return_tensors="pt",
                            padding="max_length",
                            truncation=True,
                            max_length=77).to(device)
    outputs = clip_model(**inputs)
    logits_per_image = outputs.logits_per_image
    clip_score = logits_per_image.item()

    chairf = calculate_chair_metrics(generated_caption, image_objects)
    print(f'spice_score: {spice_score}, clip_score: {clip_score}, chairf: {chairf}')

    return spice_score, clip_score, chairf

In [ ]:
val_results = []

with torch.no_grad():
    for batch in tqdm(val_dataloader, desc="Calculating SPICE, CLIPScore, and CHAIRf"):
        image_urls = batch['image_url']
        image_ids = batch['Image_ID']
        reference_captions = batch['reference_caption']

        images = []
        for image_url in image_urls:
            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
            images.append(image)

        image_objects_batch = [
            extract_objects_from_caption(ref_caption, evaluation_objects)
            for ref_caption in reference_captions
        ]

        inputs = processor(images=images, text=['Describe this image in detail.'] * len(images), return_tensors="pt").to(device)

        generated_ids = model.generate(**inputs,
                                       do_sample=True,
                                       num_beams=5,
                                       max_length=256,
                                       min_length=32,
                                       top_p=0.9,
                                       repetition_penalty=1.5,
                                       length_penalty=1.0,
                                       temperature=1)

        generated_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)
        generated_captions = [caption.strip() for caption in generated_captions]

        for image_id, generated_caption, reference_caption, image, image_objects in zip(
            image_ids, generated_captions, reference_captions, images, image_objects_batch
        ):
            spice_score, clip_score, chairf = calculate_metrics(
                image, generated_caption, reference_caption, image_objects
            )

            val_results.append({
                "Image_ID": image_id,
                "generated_caption": generated_caption,
                "reference_caption": reference_caption,
                "spice_score": spice_score,
                "clip_score": clip_score,
                "chairf": chairf,
            })

In [ ]:
results_df = pd.DataFrame(val_results)

average_spice = results_df["spice_score"].mean()
average_clip_score = results_df["clip_score"].mean()
average_chairf = results_df["chairf"].mean()

print(f"Average SPICE Score: {average_spice:.4f}")
print(f"Average CLIPScore: {average_clip_score:.4f}")
print(f"Average CHAIRf Score: {average_chairf:.4f}")

def calculate_custom_score(spice_score, clip_score, chairf):
    custom_score = (0.4 * spice_score) + (0.2 * (clip_score / 250)) + (0.4 * chairf)
    return custom_score

results_df["custom_score"] = results_df.apply(
    lambda row: calculate_custom_score(
        row["spice_score"], row["clip_score"], row["chairf"]
    ), axis=1
)

average_custom_score = results_df["custom_score"].mean()
print(f"Average Custom Score: {average_custom_score:.4f}")

## 11. 결과 저장 및 평균 점수 출력
평균 점수를 출력하고 결과를 CSV 파일로 저장하여 제출에 활용할 수 있도록 합니다.

In [ ]:
submission_data = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Generating Captions"):
        image_urls = batch['image_url']
        image_ids = batch['Image_ID']

        images = []
        for image_url in image_urls:
            response = requests.get(image_url)
            image = Image.open(BytesIO(response.content)).convert('RGB')
            images.append(image)

        inputs = processor(images=images, text=['Describe this image in detail.'] * len(images), return_tensors="pt").to(device)

        generated_ids = model.generate(**inputs,
                                       do_sample=True,
                                       num_beams=5,
                                       max_length=256,
                                       min_length=32,
                                       top_p=0.9,
                                       repetition_penalty=1.5,
                                       length_penalty=1.0,
                                       temperature=1)

        generated_captions = processor.batch_decode(generated_ids, skip_special_tokens=True)
        generated_captions = [caption.strip() for caption in generated_captions]

        for image_id, generated_caption in zip(
            image_ids, generated_captions
        ):
            submission_data.append({
                "Image_ID": image_id,
                "generated_caption": generated_caption
            })

In [ ]:
submission_df = pd.DataFrame(submission_data)
submission_df.to_csv("submission.csv", index=False)

print("Submission file 'submission.csv' created successfully.")

# 예시 1 - 프롬프트 엔지니어링 (Prompt Engineering)

## 1. 이미지 불러오기 및 전처리
주어진 이미지 URL을 통해 이미지를 불러오고, `processor`를 사용하여 모델의 입력 형식에 맞게 전처리합니다. 이 과정에서 이미지를 RGB로 변환하고, 입력 텐서를 생성하여 모델이 이해할 수 있도록 처리합니다.

In [ ]:
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/LAVIS/assets/merlion.png'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
raw_image = raw_image.resize((596, 437))
display(raw_image)

## 2. 빔 서치와 누클리어스 샘플링을 통한 캡션 생성

### 빔 서치 (Beam Search)
빔 서치는 여러 후보 캡션 경로를 동시에 탐색하며 최적의 문장을 생성하는 방식입니다. `num_beams=5`로 설정해 5개의 경로를 탐색합니다. 이 방법은 높은 품질의 캡션을 생성하는 데 유리합니다.

In [ ]:
inputs = processor(images=raw_image, text='Please describe this image briefly.', return_tensors="pt").to(device)
generated_ids = model.generate(**inputs,
                               num_beams=5,
                               max_length=50)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(f"생성된 캡션 (빔 서치): {generated_text}")

### 누클리어스 샘플링 (Nucleus Sampling)
누클리어스 샘플링은 특정 누적 확률 (top-p) 이하의 단어들 중에서만 다음 단어를 선택하는 방식입니다. `top_p=0.9`로 설정해, 상위 90% 확률에 해당하는 단어들 중에서만 다음 단어를 샘플링하여 다양한 문장 생성을 유도합니다.

In [ ]:
generated_sequences = []
for _ in range(3):
    generated_ids = model.generate(
        **inputs,
        do_sample=True,
        top_p=0.9,
        max_length=50
    )
    generated_sequences.append(generated_ids)

print("생성된 캡션 (누클리어스 샘플링):")
for i, generated_ids in enumerate(generated_sequences):
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    print(f"캡션 {i + 1}: {generated_text}")

## 3. 이미지 기반 Q&A
주어진 프롬프트와 함께 이미지를 입력으로 사용하여 질문에 대한 답변을 생성합니다. 프롬프트에 질문-응답 쌍을 기반으로 문맥을 생성하여, 모델이 문맥에 맞는 답변을 생성하도록 합니다.

In [ ]:
prompt = "Question: which city is this? Answer: singapore. Question: why?"
inputs = processor(
    images=raw_image,
    text=prompt,
    return_tensors="pt"
).to(device)

generated_ids = model.generate(**inputs, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(f"생성된 응답: {generated_text}")

## 4. 프롬프트 확장 및 이미지 기반 질문 응답
문맥을 제공하여 모델이 좀 더 정교한 응답을 생성하도록 합니다. 여기서 주어진 질문-응답 쌍들을 바탕으로 프롬프트를 구성하고, 이어지는 질문에 대해 모델이 답변하도록 합니다.

In [ ]:
context = [
    ("which city is this?", "singapore"),
    ("why?", "it has a statue of a merlion"),
]
question = "where is the name merlion coming from?"
template = "Question: {} Answer: {}."

prompt = " ".join([template.format(q, a) for q, a in context]) + f" Question: {question} Answer:"
print(f"프롬프트: {prompt}")

inputs = processor(
    images=raw_image,
    text=prompt,
    return_tensors="pt"
).to(device)

generated_ids = model.generate(**inputs, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
print(f"생성된 응답: {generated_text}")

# 예시 2 - 간단한 학습 자유 (training-free) 방법

아래 예시는 참가자들의 training-free 방법론에 대한 이해를 돕고자 간단하게 구상한 것입니다. 실제로 효과가 검증이 되어 있지 않는 방법임을 참고하여 주시길 바랍니다. 여러분들의 창의적인 아이디어를 기대하고 있습니다!

이 코드는 이미지 캡션 생성을 위해 `generate_with_boost` 함수를 사용해 이미지 및 텍스트 데이터를 처리하고, 다양한 성능 지표를 계산하여 평가합니다. 특히, 임베딩 벡터의 요소별로 평균을 기준으로 가중치를 조정하여 생성된 캡션의 품질에 영향을 줄 수 있습니다.



## 1. Boosted Embeddings 설정
### `generate_with_boost` 함수
이 함수는 모델의 임베딩 가중치를 조정하여 입력 텍스트에 대해 더 집중된 응답을 생성할 수 있도록 합니다. 임베딩 가중치는 전체 임베딩 벡터의 평균을 기준으로 증감되며, **평균보다 큰 값에는 `boost_factor`를 곱하고, 작은 값에는 나눕니다**. 이를 통해 특정 임베딩 벡터의 영향을 강조하거나 약화시킵니다.

- `mean_embedding_value`는 임베딩 벡터의 평균값입니다.
- `torch.where`를 사용하여 **임베딩 벡터 요소가 평균보다 클 경우 `boost_factor`를 곱하고, 작을 경우 나눕니다**.

In [ ]:
import torch
from tqdm import tqdm

def generate_with_boost(model, inputs, boost_factor=1.5):
    with torch.no_grad():
        embeddings = model.get_input_embeddings()
        mean_embedding_value = embeddings.weight.mean()

        boosted_embeddings_weight = torch.where(
            embeddings.weight > mean_embedding_value,
            embeddings.weight * boost_factor,
            embeddings.weight / boost_factor
        )

        model.get_input_embeddings().weight.data = boosted_embeddings_weight

        generated_ids = model.generate(**inputs,
                                do_sample=True,
                                num_beams=5,
                                max_length=256,
                                min_length=32,
                                top_p=0.9,
                                repetition_penalty=1.5,
                                length_penalty=1.0,
                                temperature=1)
        generated_caption = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()

    return generated_caption

## 2. 데이터 처리 및 캡션 생성
이미지 데이터와 참조 캡션을 기반으로 이미지의 객체 정보를 추출하고, 이를 통해 **임베딩 가중치가 조정된 모델로 캡션을 생성**합니다. 결과 캡션과 성능 지표를 저장해 평가할 수 있도록 합니다.

In [ ]:
val_results = []

with torch.no_grad():
    for idx, row in tqdm(val_df.iloc[:3].iterrows(), total=len(val_df.iloc[:3]), desc="Calculating SPICE, CLIPScore, and CHAIRf"):
        image_url = row['url']
        image_id = row['Image_ID']
        reference_caption = row['Paragraph']

        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert('RGB')
        image_objects = extract_objects_from_caption(reference_caption, evaluation_objects)

        inputs = processor(images=image, text='Describe this image in detail.', return_tensors="pt").to(device)
        generated_caption = generate_with_boost(model, inputs, boost_factor=1.5)
        print(generated_caption)

        spice_score, clip_score, chairf = calculate_metrics(
            image, generated_caption, reference_caption, image_objects
        )

        val_results.append({
            "Image_ID": image_id,
            "generated_caption": generated_caption,
            "reference_caption": reference_caption,
            "spice_score": spice_score,
            "clip_score": clip_score,
            "chairf": chairf
        })

In [ ]:
results_df = pd.DataFrame(val_results)

average_spice = results_df["spice_score"].mean()
average_clip_score = results_df["clip_score"].mean()
average_chairf = results_df["chairf"].mean()

print(f"Average SPICE Score: {average_spice:.4f}")
print(f"Average CLIPScore: {average_clip_score:.4f}")
print(f"Average CHAIRf Score: {average_chairf:.4f}")

def calculate_custom_score(spice_score, clip_score, chairf):
    custom_score = (0.4 * spice_score) + (0.2 * (clip_score / 250)) + (0.4 * chairf)
    return custom_score

results_df["custom_score"] = results_df.apply(
    lambda row: calculate_custom_score(
        row["spice_score"], row["clip_score"], row["chairf"]
    ), axis=1
)

average_custom_score = results_df["custom_score"].mean()
print(f"Average Custom Score: {average_custom_score:.4f}")